In [125]:
import sys 
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
# sys.path.append('/users/jiajiaz/.conda/envs/mybrainiak/lib/python3.6/site-packages')
sys.path.append('../Multivariate_analyses_Daphne/')
import os 
import h5py
import glob
import time
import numpy as np
import pandas as pd 

from nilearn import datasets, image
from nilearn import surface
from nilearn import plotting
from nilearn import input_data

from nilearn.input_data import NiftiMasker, NiftiLabelsMasker
from nibabel.affines import apply_affine
import nibabel as nib
import time

from brainiak import image, io
from brainiak.isc import isc, isfc, permutation_isc
from brainiak.isc import compute_summary_statistic
from brainiak.searchlight.searchlight import Searchlight
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d 
import seaborn as sns 
import pandas as pd
from importlib import reload 
import scipy.io as sio
from scipy import stats
from numpy.linalg import inv

# import own functions
import utils
reload(utils)

sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")

# Load data

In [3]:
BOLD_path = '/ncf/gershman/Lab/scripts/matlab/VGDL_fMRI/mat/'
path = './data/'
filename = 'BOLD_subj1_nosmooth.mat'

data = h5py.File(BOLD_path+filename,'r') 

print(data.keys())
print(data['B'].value.shape)

<KeysViewHDF5 ['#refs#', 'B', 'Vmask', 'block', 'game', 'instance', 'interaction_change_flag', 'interaction_change_flag_cnt', 'level', 'mask', 'run', 'runs', 'termination_change_flag', 'termination_change_flag_cnt', 'theory_change_flag', 'theory_change_flag_cnt']>
(179595, 1698)


In [55]:
# mask_nii is the functional mask, this selects the brain voxels
mask_nii_NS = nib.load(os.path.join(path, 'mask_nosmooth.nii')) 
mask_nii = nib.load(os.path.join(path, 'mask.nii')) 
print(mask_nii.shape)
# inverse of the affine matrix: mni2cor
inv_affine = inv(mask_nii_NS.affine) # get the transformation matrix

# load mask and get voxel coordinates
# mask_arr = np.load(path+'mask_arr.npy') 
mask_mat = data['mask'].value
coords_mat = np.array(np.where(mask_mat == 1)) # so need one set of voxel coordinates for all
coords_mat[[0, 2]] = coords_mat[[2, 0]] # exchange the rows

# residuals
# R = np.load(path+'residuals_sub1.npy')
# R = np.swapaxes(R,1,0)

(79, 95, 79)


In [54]:
print(BOLD1.shape)
print(len(np.where(mask_nii_NS.dataobj)[0]))

(1698, 179595)
179595


In [18]:
BOLD1 = pd.DataFrame(np.swapaxes(data['B'].value,1,0)) # store residuals for subject
game1 = pd.DataFrame(np.swapaxes(data['game'].value,1,0), columns=["game"]).shift(periods=3)
level1 = pd.DataFrame(np.swapaxes(data['level'].value,1,0), columns=["level"]).shift(periods=3)
tcf1 = pd.DataFrame(np.swapaxes(data['theory_change_flag'].value,1,0), columns=["tcf"]).shift(periods=3)
tcfc1 = pd.DataFrame(np.swapaxes(data['theory_change_flag_cnt'].value,1,0), columns=["tcfc"]).shift(periods=3)
# np.save(path+'residuals_sub1', residuals_sub1)

## Save the theory change flag, theory change flag cnt as well

In [19]:
df = pd.concat([BOLD1, game1, level1, tcf1, tcfc1], axis=1)

In [31]:
print(df.game.unique())
print(df.level.unique())

[nan  1.  4.  3.  5.  2.  6.]
[nan  1.  2.  3.  4.  5.  6.  7.  8.  9.]


## Reorder data based on game and level. Fill in NaN values

In [70]:
print(cur_df1.loc[1208,0])


84.72139466476993


In [ ]:
num_TR = 58
vol4D = mask_nii.shape+(num_TR,)
coords = tuple(coords_mat)
num_voxel = len(np.where(mask_nii_NS.dataobj)[0])
all_vol = []
all_df = []
all_tcf = []
all_tcfc = []
num_subj = 1
for subj in range(num_subj):
    all_df.append(pd.DataFrame())
    all_vol.append(np.zeros(vol4D))
# For each game+level, get the start and end
for game_id in range(1,2):
        for level_id in range(1,3):
            for subj in range(1):
                isc_df = all_df[subj]
                # TODO change the below to have an array of df from raw data
                cur_df = df[(df['game']==game_id) & (df['level']==level_id)]
                # TODO In "averaging" case, need to look across subjects to find shortest cur_df num rows
                isc_df = pd.concat([isc_df, cur_df])
                all_df[subj] = isc_df

In [95]:
for subj in range(num_subj):
    isc_df = all_df[subj]
    isc_vol = all_vol[subj]
    isc_df = isc_df.reset_index(drop=True)
    isc_df['tcf'] = isc_df['tcf'].fillna(value=0)
    isc_df['tcfc'] = isc_df['tcfc'].fillna(value=0)
    print(isc_df.shape[0])
    for TR in range(isc_df.shape[0]):
        for j in range(num_voxel):
            isc_vol[(coords[0][j], coords[1][j], coords[2][j], TR)] = isc_df.loc[TR, j]
    all_tcfc.append(isc_df['tcfc'])
    all_tcf.append(isc_df['tcf'])
    print(isc_df.head(10))
    all_df[subj] = isc_df 
    all_vol[subj] = isc_vol

58
           0          1          2          3          4          5  \
0  72.472996  75.539245  78.298870  80.270030  80.708066  78.999727   
1  78.320771  83.292476  87.388109  89.600189  89.162153  85.811181   
2  74.663174  77.817030  80.401441  82.175485  82.438306  80.423342   
3  74.925995  78.780709  81.956467  83.905726  83.796217  80.992789   
4  75.210718  80.708066  85.241734  87.694734  87.147189  83.314378   
5  75.232620  79.788191  83.358181  85.307440  85.132226  82.569717   
6  75.495442  79.875798  83.533395  85.526458  85.132226  82.087878   
7  72.779621  75.364031  77.970343  80.204325  81.124199  79.744387   
8  74.049924  77.488504  80.335735  82.175485  82.306896  80.051012   
9  76.656236  81.934565  85.811181  87.629029  86.884368  83.533395   

           6          7          8          9  ...      179589      179590  \
0  75.561147  77.488504  79.021628  80.598557  ...  161.372328  146.829545   
1  79.043530  82.613521  85.635966  88.242279  ...  160.299

In [105]:
print(all_vol[0].shape)
print(np.array(all_tcfc[0]))
print(all_tcfc[0][all_tcfc[0] > 0].index.values)
print(all_tcfc[0].shape[0])

(79, 95, 79, 249)
[2. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 3. 0. 0. 0. 0. 0. 3. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
[ 0  1  3 31 32 38 56]
58


# Translate from MNI space for searchlight

In [142]:
# theory ROIs (model-based), encoding theory
# all MNI coordinates!
R_IFG_Tri = [42, 28, 26]
L_Insula = [-30, 28, 2]
R_DMPFC = [6, 38, 40]
L_IFG_Tri = [-50, 44, 12]
L_MTG = [-64, -50, 4]
R_MTG = [58, -36, 8]

# Theory_change_flag ROIs, updating theory
R_IFG_Oper_U = [48, 12, 28]
L_PPC_U = [-56, -32, 46]
R_IFG_Tri_U = [52, 38, 16]
R_AG_U = [32, -60, 34]
L_Fusiform_U = [-40, -58, -12]
L_IFG_Oper_U = [-42, 4, 28]
R_PHC_U = [26, -42, -8]

# create dictionary to map points to roi names
roi_dict = {'right inferior frontal gyrus (Tri)':R_IFG_Tri, 'left_insula':L_Insula, 'right dorsalmedial PFC':R_DMPFC,
            'left inferior frontal gyrus (Tri)':L_IFG_Tri, 'left middle temporal gyrus':L_MTG, 'right middle temporal gyrus':R_MTG
           }

brain_mask = np.array(mask_nii.dataobj)
roi_mask = np.zeros(brain_mask.shape)
print(roi_mask[roi_mask==1].shape)
native_roi_coords = []
for mni_coords in roi_dict:
    native_coords = apply_affine(aff=inv_affine, pts=roi_dict[mni_coords]).astype(int) # from mni2cor
#     native_roi_coords += native_coords.astype(int)
    roi_mask[tuple(native_coords)] = 1
print(roi_mask[roi_mask==1].shape)

(0,)
(6,)


# Using searchlight to get an r value for each region

In [148]:
# Preset the variables
data = all_vol
mask = roi_mask
bcvar = all_tcfc
sl_rad = 8
max_blk_edge = 5
pool_size = 1
begin_time = time.time()
# Create the searchlight object
sl = Searchlight(sl_rad=sl_rad,max_blk_edge=max_blk_edge)
print("Setup searchlight inputs")
print("Number of subjects: " + str(len(data)))
print("Input data shape: " + str(data[0].shape))
print("Input mask shape: " + str(mask.shape) + "\n")

# Distribute the information to the searchlights (preparing it to run)
sl.distribute(data, mask)
# Data that is needed for all searchlights is sent to all cores via the sl.broadcast function. 
sl.broadcast(bcvar)

Setup searchlight inputs
Number of subjects: 1
Input data shape: (79, 95, 79, 249)
Input mask shape: (79, 95, 79)



In [149]:
# Set up the kernel
def avg_corr(data, sl_mask, myrad, bcvar):
    all_r = []
    TRs = bcvar[0].shape[0]
    # Loop over subject: 
    # TODO focus on subject 0 for now
    for idx in range(len(data)):
        subj_r = []
        data4D = data[idx]
        tcfc = bcvar[idx]
        bolddata_sl = data4D.reshape(sl_mask.shape[0] * sl_mask.shape[1] * sl_mask.shape[2], data[0].shape[3]).T
        tcf = bcvar[idx][bcvar[idx] > 0].index.values
        for flag_idx in tcf:
            # Ignore the first 3 TRs, for each game level 1. So we are assuming only 1 game here. 
            # And Ignore flags at the end
            if flag_idx >= 3 and flag_idx < TRs-3:
                before_idx = flag_idx - 3
                after_idx = flag_idx + 3
                flag_r = np.corrcoef(bolddata_sl[before_idx], bolddata_sl[after_idx])
                subj_r.append(flag_r)
        all_r.append(np.mean(subj_r))
    
    return all_r

In [150]:
# Execute searchlight on 8 subjects
print("Begin Searchlight\n")
sl_result_allsubj = sl.run_searchlight(avg_corr, pool_size=pool_size)
print("End Searchlight\n")
print([i for i in sl_result_allsubj[mask==1]])

Begin Searchlight

End Searchlight

[[0.9999843977712546], [0.9999639900977979], [0.9999374001501992], [0.9998622126815376], [0.9999822447311606], None]


In [151]:
print(sl_result_allsubj[roi_mask==1])

[list([0.9999843977712546]) list([0.9999639900977979])
 list([0.9999374001501992]) list([0.9998622126815376])
 list([0.9999822447311606]) None]


[1. 1. 1. ... 1. 1. 1.]
